# This Notebook trains an [API-Net](https://github.com/PeiqinZhuang/API-Net) on LV data set for *main_zipper_pull*. For more details of implementation please refer to each cell.

In [ ]:
#In this cell some utility functions are implemented. 

import torch

def save_checkpoint(state, is_best, filename='checkpoint_main-zipper-pull.pth.tar'):
    torch.save(state, '/content/drive/MyDrive/'+filename)
    if is_best:
        shutil.copyfile('/content/drive/MyDrive/'+filename, '/content/drive/MyDrive/main-zipper-pull_model_best.pth.tar')


class AverageMeter(object):
    """
    Keeps track of most recent, average, sum, and count of a metric.
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count



def accuracy(scores, targets, k):
    """
    Computes top-k accuracy, from predicted and true labels.

    :param scores: scores from the model
    :param targets: true labels
    :param k: k in top-k accuracy
    :return: top-k accuracy
    """

    batch_size = targets.size(0)
    _, ind = scores.topk(k, 1, True, True)
    correct = ind.eq(targets.view(-1, 1).expand_as(ind))
    correct_total = correct.view(-1).float().sum()  # 0D tensor
    return correct_total.item() * (100.0 / batch_size)


In [ ]:
# Importing required packages.
from torch import nn

import torchvision
from torchvision import models
import torchvision.transforms as transforms

import torch.utils.data
from torch.utils.data import Dataset
from torch.utils.data.sampler import BatchSampler

import torch.nn.functional as F
import torch.optim

from PIL import Image
import numpy as np
import os
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch.backends.cudnn as cudnn


In [ ]:
# In this cell we implement a custom dataset that inherits from pytorch's dataset. 
# The RandomDataset is used for validation set and BatchDataset is for training set
# They read the images and their labels from a .txt file that is formatted as follows
# /path/to/imagefile/.jpg label

def default_loader(path):
    try:
        img = Image.open(path).convert('RGB')
    except:
        with open('read_error.txt', 'a') as fid:
            fid.write(path + '\n')
        return Image.new('RGB', (224, 224), 'white')
    return img


class RandomDataset(Dataset):
    def __init__(self, transform=None, dataloader=default_loader):
        self.transform = transform
        self.dataloader = dataloader

        with open('/content/drive/MyDrive/LV_data/main_zipper_pull/val/val.txt', 'r') as fid:
            self.imglist = fid.readlines()

        self.labels = []
        for line in self.imglist:
            image_path, label = line.strip().split()
            self.labels.append(int(label))
        self.labels = np.array(self.labels)
        self.labels = torch.LongTensor(self.labels)

    def __getitem__(self, index):
        image_name, label = self.imglist[index].strip().split()
        image_path = image_name
        img = self.dataloader(image_path)
        img = self.transform(img)
        label = int(label)
        label = torch.LongTensor([label])

        return img, label

    def __len__(self):
        return len(self.imglist)


class BatchDataset(Dataset):
    def __init__(self, transform=None, dataloader=default_loader):
        self.transform = transform
        self.dataloader = dataloader

        with open('/content/drive/MyDrive/LV_data/main_zipper_pull/train/train.txt', 'r') as fid:
            self.imglist = fid.readlines()

        self.labels = []
        for line in self.imglist:
            #print(i)
            image_path, label = line.strip().split()
            self.labels.append(int(label))
        self.labels = np.array(self.labels)
        self.labels = torch.LongTensor(self.labels)

    def __getitem__(self, index):
        image_name, label = self.imglist[index].strip().split()
        image_path = image_name
        img = self.dataloader(image_path)
        img = self.transform(img)
        label = int(label)
        label = torch.LongTensor([label])

        return img, label

    def __len__(self):
        return len(self.imglist)


class BalancedBatchSampler(BatchSampler):
    def __init__(self, dataset, n_classes, n_samples):
        self.labels = dataset.labels
        self.labels_set = list(set(self.labels.numpy()))
        self.label_to_indices = {label: np.where(self.labels.numpy() == label)[0]
                                 for label in self.labels_set}
        for l in self.labels_set:
            np.random.shuffle(self.label_to_indices[l])
        self.used_label_indices_count = {label: 0 for label in self.labels_set}
        self.count = 0
        self.n_classes = n_classes
        self.n_samples = n_samples
        self.dataset = dataset
        self.batch_size = self.n_samples * self.n_classes

    def __iter__(self):
        self.count = 0
        while self.count + self.batch_size < len(self.dataset):
            classes = np.random.choice(self.labels_set, self.n_classes, replace=False)
            indices = []
            for class_ in classes:
                indices.extend(self.label_to_indices[class_][
                               self.used_label_indices_count[class_]:self.used_label_indices_count[
                                                                         class_] + self.n_samples])
                self.used_label_indices_count[class_] += self.n_samples
                if self.used_label_indices_count[class_] + self.n_samples > len(self.label_to_indices[class_]):
                    np.random.shuffle(self.label_to_indices[class_])
                    self.used_label_indices_count[class_] = 0
            yield indices
            self.count += self.n_classes * self.n_samples

    def __len__(self):
        return len(self.dataset) // self.batch_size


In [ ]:
# This is the actual implementation of API-Net. For details please see the corresponding paper 
# in official github page in above link 
def pdist(vectors):
    distance_matrix = -2 * vectors.mm(torch.t(vectors)) + vectors.pow(2).sum(dim=1).view(1, -1) + vectors.pow(2).sum(
        dim=1).view(-1, 1)
    return distance_matrix

class API_Net(nn.Module):
    def __init__(self):
        super(API_Net, self).__init__()

        resnet101 = models.resnet101(pretrained=True)
        layers = list(resnet101.children())[:-2]

        self.conv = nn.Sequential(*layers)
        self.avg = nn.AvgPool2d(kernel_size=14, stride=1)
        self.map1 = nn.Linear(2048 * 2, 512)
        self.map2 = nn.Linear(512, 2048)
        self.fc = nn.Linear(2048, 2)
        self.drop = nn.Dropout(p=0.5)
        self.sigmoid = nn.Sigmoid()


    def forward(self, images, targets=None, flag='train'):
        conv_out = self.conv(images)
        pool_out = self.avg(conv_out).squeeze()

        if flag == 'train':
            intra_pairs, inter_pairs, \
                    intra_labels, inter_labels = self.get_pairs(pool_out, targets)

            features1 = torch.cat([pool_out[intra_pairs[:, 0]], pool_out[inter_pairs[:, 0]]], dim=0)
            features2 = torch.cat([pool_out[intra_pairs[:, 1]], pool_out[inter_pairs[:, 1]]], dim=0)
            labels1 = torch.cat([intra_labels[:, 0], inter_labels[:, 0]], dim=0)
            labels2 = torch.cat([intra_labels[:, 1], inter_labels[:, 1]], dim=0)


            mutual_features = torch.cat([features1, features2], dim=1)
            map1_out = self.map1(mutual_features)
            map2_out = self.drop(map1_out)
            map2_out = self.map2(map2_out)


            gate1 = torch.mul(map2_out, features1)
            gate1 = self.sigmoid(gate1)

            gate2 = torch.mul(map2_out, features2)
            gate2 = self.sigmoid(gate2)

            features1_self = torch.mul(gate1, features1) + features1
            features1_other = torch.mul(gate2, features1) + features1

            features2_self = torch.mul(gate2, features2) + features2
            features2_other = torch.mul(gate1, features2) + features2

            logit1_self = self.fc(self.drop(features1_self))
            logit1_other = self.fc(self.drop(features1_other))
            logit2_self = self.fc(self.drop(features2_self))
            logit2_other = self.fc(self.drop(features2_other))

            return logit1_self, logit1_other, logit2_self, logit2_other, labels1, labels2

        elif flag == 'val':
            return self.fc(pool_out)


    def get_pairs(self, embeddings, labels):
        distance_matrix = pdist(embeddings).detach().cpu().numpy()

        labels = labels.detach().cpu().numpy().reshape(-1,1)
        num = labels.shape[0]
        dia_inds = np.diag_indices(num)
        lb_eqs = (labels == labels.T)
        lb_eqs[dia_inds] = False
        dist_same = distance_matrix.copy()
        dist_same[lb_eqs == False] = np.inf
        intra_idxs = np.argmin(dist_same, axis=1)

        dist_diff = distance_matrix.copy()
        lb_eqs[dia_inds] = True
        dist_diff[lb_eqs == True] = np.inf
        inter_idxs = np.argmin(dist_diff, axis=1)

        intra_pairs = np.zeros([embeddings.shape[0], 2])
        inter_pairs  = np.zeros([embeddings.shape[0], 2])
        intra_labels = np.zeros([embeddings.shape[0], 2])
        inter_labels = np.zeros([embeddings.shape[0], 2])
        for i in range(embeddings.shape[0]):
            intra_labels[i, 0] = labels[i]
            intra_labels[i, 1] = labels[intra_idxs[i]]
            intra_pairs[i, 0] = i
            intra_pairs[i, 1] = intra_idxs[i]

            inter_labels[i, 0] = labels[i]
            inter_labels[i, 1] = labels[inter_idxs[i]]
            inter_pairs[i, 0] = i
            inter_pairs[i, 1] = inter_idxs[i]

        intra_labels = torch.from_numpy(intra_labels).long().to(device)
        intra_pairs = torch.from_numpy(intra_pairs).long().to(device)
        inter_labels = torch.from_numpy(inter_labels).long().to(device)
        inter_pairs = torch.from_numpy(inter_pairs).long().to(device)

        return intra_pairs, inter_pairs, intra_labels, inter_labels


In [ ]:
# Some hyper parameters. If you try to resume the training then pytorch will load
# the trained weight defined in variable "resume".
workers=2
epochs=100
start_epoch=0
lr=0.01
momentum=0.9
weight_decay=5e-4
print_freq=20
evaluate_freq=20
resume='/content/drive/MyDrive/checkpoint_main-zipper-pull.pth.tar'
n_classes=2
n_samples=8

In [ ]:
# The function for training the model. notice that the "validate" function defined in next cell
# is called inside "train" function
def train(train_loader, model, criterion, optimizer_conv, scheduler_conv, optimizer_fc, scheduler_fc, epoch, step):
    best_prec1 = 0
    batch_time = AverageMeter()
    data_time = AverageMeter()
    softmax_losses = AverageMeter()
    rank_losses = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    
    # switch to train mode
    end = time.time()
    rank_criterion = nn.MarginRankingLoss(margin=0.05)
    softmax_layer = nn.Softmax(dim=0).to(device)

    for i, (input, target) in enumerate(train_loader):
        model.train()

        # measure data loading time
        data_time.update(time.time() - end)
        input_var = input.to(device)
        target_var = target.to(device).squeeze()

        # compute output
        logit1_self, logit1_other, logit2_self, logit2_other, labels1, labels2 = model(input_var, target_var,flag='train')
        batch_size = logit1_self.shape[0]
        labels1 = labels1.to(device)
        labels2 = labels2.to(device)

        self_logits = torch.zeros(2 * batch_size, 2).to(device)
        other_logits = torch.zeros(2 * batch_size, 2).to(device)
        self_logits[:batch_size] = logit1_self
        self_logits[batch_size:] = logit2_self
        other_logits[:batch_size] = logit1_other
        other_logits[batch_size:] = logit2_other

        # compute loss
        logits = torch.cat([self_logits, other_logits], dim=0)
        targets = torch.cat([labels1, labels2, labels1, labels2], dim=0)
    
        softmax_loss = criterion(logits, targets)

        self_scores = softmax_layer(self_logits)[torch.arange(2 * batch_size).to(device).long(),
                                                 torch.cat([labels1, labels2], dim=0)]
        other_scores = softmax_layer(other_logits)[torch.arange(2 * batch_size).to(device).long(),
                                                   torch.cat([labels1, labels2], dim=0)]
        flag = torch.ones([2 * batch_size, ]).to(device)
        rank_loss = rank_criterion(self_scores, other_scores, flag)

        loss = softmax_loss + rank_loss

        # measure accuracy and record loss
        prec1 = accuracy(logits, targets, 1)
        #prec2 = accuracy(logits, targets, 2)
        losses.update(loss.item(), 2 * batch_size)
        softmax_losses.update(softmax_loss.item(), 4 * batch_size)
        rank_losses.update(rank_loss.item(), 2 * batch_size)
        top1.update(prec1, 4 * batch_size)
        #top2.update(prec2, 4 * batch_size)

        # compute gradient and do SGD step
        optimizer_conv.zero_grad()
        optimizer_fc.zero_grad()
        loss.backward()
        if epoch >= 8:
            optimizer_conv.step()
        optimizer_fc.step()
        scheduler_conv.step()
        scheduler_fc.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Time: {time}\nStep: {step}\t Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'SoftmaxLoss {softmax_loss.val:.4f} ({softmax_loss.avg:.4f})\t'
                  'RankLoss {rank_loss.val:.4f} ({rank_loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                epoch, i, len(train_loader), batch_time=batch_time,
                data_time=data_time, loss=losses, softmax_loss=softmax_losses, rank_loss=rank_losses,
                top1=top1, step=step, time=time.asctime(time.localtime(time.time()))))

        if i == len(train_loader) - 1:
            val_dataset = RandomDataset(transform=transforms.Compose([
                transforms.Resize([512, 512]),
                transforms.CenterCrop([448, 448]),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)
                )]))
            val_loader = torch.utils.data.DataLoader(
                val_dataset, batch_size=batch_size, shuffle=False,
                num_workers=workers, pin_memory=True)
            prec1 = validate(val_loader, model, criterion)

            # remember best prec@1 and save checkpoint
            is_best = prec1 > best_prec1
            best_prec1 = max(prec1, best_prec1)
            save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'best_prec1': best_prec1,
                'optimizer_conv': optimizer_conv.state_dict(),
                'optimizer_fc': optimizer_fc.state_dict(),
            }, is_best)

        step = step + 1
    return step

In [ ]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter()
    softmax_losses = AverageMeter()
    top1 = AverageMeter()
    #top2 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    end = time.time()

    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):

            input_var = input.to(device)
            target_var = target.to(device).squeeze()

            # compute output
            logits = model(input_var, targets=None, flag='val')
            #print(logits)
            #print(target_var)
            softmax_loss = criterion(logits, target_var)
            

            prec1 = accuracy(logits, target_var, 1)
            #prec2 = accuracy(logits, target_var, 2)
            softmax_losses.update(softmax_loss.item(), logits.size(0))
            top1.update(prec1, logits.size(0))
            #top2.update(prec2, logits.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:
                print('Time: {time}\nTest: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'SoftmaxLoss {softmax_loss.val:.4f} ({softmax_loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                    i, len(val_loader), batch_time=batch_time, softmax_loss=softmax_losses,
                    top1=top1, time=time.asctime(time.localtime(time.time()))))
        print(' * Prec@1 {top1.avg:.3f}'.format(top1=top1))

    return top1.avg


In [ ]:
#The main cell. if you've done with above cells you can start training by running this cell.
torch.manual_seed(20)
torch.cuda.manual_seed_all(20)
np.random.seed(25)
epochs=100
# create model
model = API_Net()
model = model.to(device)
model.conv = nn.DataParallel(model.conv)

# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer_conv = torch.optim.SGD(model.conv.parameters(), lr,
                                  momentum=momentum,
                                  weight_decay=weight_decay)

fc_parameters = [value for name, value in model.named_parameters() if 'conv' not in name]
optimizer_fc = torch.optim.SGD(fc_parameters, lr,
                                momentum=momentum,
                                weight_decay=weight_decay)
if resume:
    if os.path.isfile(resume):
        print('loading checkpoint {}'.format(resume))
        checkpoint = torch.load(resume)
        start_epoch = checkpoint['epoch']
        best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer_conv.load_state_dict(checkpoint['optimizer_conv'])
        optimizer_fc.load_state_dict(checkpoint['optimizer_fc'])
        print('loaded checkpoint {}(epoch {})'.format(resume, checkpoint['epoch']))
    else:
        print('no checkpoint found at {}'.format(resume))

cudnn.benchmark = True
# Data loading code
train_dataset = BatchDataset(transform=transforms.Compose([
    transforms.Resize([512, 512]),
    transforms.RandomCrop([448, 448]),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)
    )]))

train_sampler = BalancedBatchSampler(train_dataset, n_classes, n_samples)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_sampler, num_workers=workers, pin_memory=True)
scheduler_conv = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_conv, 100 * len(train_loader))
scheduler_fc = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_fc, 100 * len(train_loader))

step = 0
print('START TIME:', time.asctime(time.localtime(time.time())))
for epoch in range(start_epoch, epochs):
    step = train(train_loader, model, criterion, optimizer_conv, scheduler_conv, optimizer_fc, scheduler_fc, epoch, step)


loading checkpoint /content/drive/MyDrive/checkpoint_main-zipper-pull.pth.tar
loaded checkpoint /content/drive/MyDrive/checkpoint_main-zipper-pull.pth.tar(epoch 80)
START TIME: Thu Mar 31 23:56:46 2022
Time: Thu Mar 31 23:56:47 2022
Step: 0	 Epoch: [80][0/204]	Time 1.579 (1.579)	Data 1.065 (1.065)	Loss 0.2291 (0.2291)	SoftmaxLoss 0.1789 (0.1789)	RankLoss 0.0501 (0.0501)	Prec@1 84.375 (84.375)
Time: Thu Mar 31 23:56:58 2022
Step: 20	 Epoch: [80][20/204]	Time 0.596 (0.573)	Data 0.099 (0.074)	Loss 0.0674 (0.1043)	SoftmaxLoss 0.0166 (0.0537)	RankLoss 0.0508 (0.0506)	Prec@1 100.000 (97.061)
Time: Thu Mar 31 23:57:08 2022
Step: 40	 Epoch: [80][40/204]	Time 0.531 (0.552)	Data 0.027 (0.053)	Loss 0.0857 (0.0970)	SoftmaxLoss 0.0345 (0.0465)	RankLoss 0.0512 (0.0505)	Prec@1 100.000 (97.961)
Time: Thu Mar 31 23:57:20 2022
Step: 60	 Epoch: [80][60/204]	Time 0.505 (0.555)	Data 0.004 (0.056)	Loss 0.0512 (0.0893)	SoftmaxLoss 0.0012 (0.0388)	RankLoss 0.0500 (0.0505)	Prec@1 100.000 (98.425)
Time: Thu Mar

# From here all cells mean for testing phase.

In [ ]:
# this class is just like RandomDataset above but we copied that with path to test dataset images.
class RandomDatasetTest(Dataset):
    def __init__(self, transform=None, dataloader=default_loader):
        self.transform = transform
        self.dataloader = dataloader

        with open('/content/drive/MyDrive/LV_data/main_closure_hardware/test/test.txt', 'r') as fid:
            self.imglist = fid.readlines()

        self.labels = []
        for line in self.imglist:
            #print(i)
            image_path, label = line.strip().split()
            self.labels.append(int(label))
        self.labels = np.array(self.labels)
        self.labels = torch.LongTensor(self.labels)

    def __getitem__(self, index):
        image_name, label = self.imglist[index].strip().split()
        image_path = image_name
        img = self.dataloader(image_path)
        img = self.transform(img)
        label = int(label)
        label = torch.LongTensor([label])

        return img, label

    def __len__(self):
        return len(self.imglist)

In [ ]:
# loading trained weight and model
model = API_Net()
model = model.to(device)
model.conv = nn.DataParallel(model.conv)
checkpoint = torch.load('/content/drive/MyDrive/main-zipper-pull_model_best.pth.tar')

model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [ ]:
# This function calculates the metrics TP, FP, TN, and FN
def confusion(scores, targets):
    """ Returns the confusion matrix for the values in the `prediction` and `truth`
    tensors, i.e. the amount of positions where the values of `prediction`
    and `truth` are
    - 1 and 1 (True Positive)
    - 1 and 0 (False Positive)
    - 0 and 0 (True Negative)
    - 0 and 1 (False Negative)
    """

    _, ind = scores.topk(1, 1, True, True)
    correct = ind.eq(targets.view(-1, 1).expand_as(ind))
    prediction = correct.view(-1).float()
    #correct_total = correct.view(-1).float().sum()  # 0D tensor
    confusion_vector = prediction / targets
    # Element-wise division of the 2 tensors returns a new tensor which holds a
    # unique value for each case:
    #   1     where prediction and truth are 1 (True Positive)
    #   inf   where prediction is 1 and truth is 0 (False Positive)
    #   nan   where prediction and truth are 0 (True Negative)
    #   0     where prediction is 0 and truth is 1 (False Negative)

    true_positives = torch.sum(confusion_vector == 1).item()
    false_positives = torch.sum(confusion_vector == float('inf')).item()
    true_negatives = torch.sum(torch.isnan(confusion_vector)).item()
    false_negatives = torch.sum(confusion_vector == 0).item()

    return true_positives, false_positives, true_negatives, false_negatives

In [ ]:
# calculating specificity and sensitivity on our test dataset
test_dataset = RandomDatasetTest(transform=transforms.Compose([
                transforms.Resize([512, 512]),
                transforms.CenterCrop([448, 448]),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)
                )]))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=6, shuffle=False,
                num_workers=workers, pin_memory=True)

criterion = nn.CrossEntropyLoss().to(device)

TP=0
TN=0
FP=0
FN=0
for input, target in test_loader:
  input_var = input.to(device)
  target_var = target.to(device).squeeze()

  logits = model(input_var, targets=None, flag='val')
  tp, fp, tn, fn = confusion(logits, target_var)

  TP += tp
  FP += fp
  TN += tn
  FN += fn
sp, se = TN / (TN+FP), TP / (TP+FN)
print('specificity: {}\t sensitivity: {}'.format(sp, se))

specificity: 0.5406976744186046	 sensitivity: 0.5247524752475248
